<a href="https://colab.research.google.com/github/mrinmoy-sadhukhan/AI-mask/blob/main/cnn(mobilenetv2)%2BSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.version
!pip install keras-applications

     |████████████████████████████████| 51kB 4.7MB/s 


In [ ]:
%cd /content/drive/My Drive/kaggle

/content/drive/My Drive/kaggle


In [ ]:
! pip install -q kaggle
from google.colab import files

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/kaggle"

In [ ]:
import zipfile

In [ ]:
#!kaggle datasets download --force -d spandanpatnaik09/face-mask-detectormask-not-mask-incorrect-mask

In [ ]:
#zip_ref = zipfile.ZipFile('/content/drive/My Drive/kaggle/face-mask-detectormask-not-mask-incorrect-mask.zip', 'r')
#zip_ref.extractall('mask_unzippeed_folder') # unzip directory
#zip_ref.close()

In [ ]:
base_path = 'mask_unzippeed_folder/dataset'
#categories = ['incorrect_mask','with_mask','without_mask']
#categories = ['with_mask','without_mask']

In [ ]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization,DepthwiseConv2D
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
import itertools

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Activation, Flatten, Dropout, concatenate, Input, Conv2D, MaxPooling2D,Conv2D, BatchNormalization, ReLU, concatenate, AveragePooling2D, Add, GlobalMaxPooling2D,Dropout, Flatten, Dense
from keras.optimizers import Adam, Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.np_utils import to_categorical

In [ ]:
INIT_LR = 1e-4
EPOCHS = 200
BS = 32

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size


# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(base_path))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	print(label,imagePath)
	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)
print(data.shape)
print(labels.shape)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#labels = to_categorical(labels)
print(labels.shape)

[INFO] loading images...
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_1.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_10.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_100.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_101.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_102.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_103.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_104.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_106.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_107.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_105.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_108.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_114.jpg
incorrect_mask mask_unzippeed_folder/dataset/incorrect_mask/aug_110.jpg
incorrect_mask mask_unzippeed_folder/datas

In [ ]:
print(data.shape)

(2079, 224, 224, 3)


In [ ]:
print(labels)

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [ ]:
(trainx, testX, trainy, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [ ]:
(trainX,valX,trainY,valY)=train_test_split(trainx,trainy,test_size=0.10,random_state=10)

In [ ]:


# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

model.summary()
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save('/content/drive/MyDrive/kaggle/scratch_train_1.model', save_format="h5")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('/content/drive/MyDrive/kaggle/plot1.png')
plt.close()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [ ]:


# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

from keras import backend as K
def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)

def fire_hee(x,first_filter,output_filter):
    x=Conv2D(first_filter,(1,1),strides=(1,1),padding='same',trainable=True)(x) //trainable false or true
    x=BatchNormalization(trainable=True)(x)
    x=Activation(relu6)(x)
    x =DepthwiseConv2D((3,3),strides=(1,1),depth_multiplier=1, padding='same',trainable=True)(x)
    x=BatchNormalization(trainable=True)(x)
    x=Activation(relu6)(x)
    x=Conv2D(output_filter,(1,1),padding='same',strides=(1,1),trainable=True)(x)
    return x


def fire_intercept(x,fire=64):
    left=Conv2D(fire,(1,1),padding='valid',strides=(1,1),trainable=True)(x)
    left=ReLU()(left)

    right=Conv2D(fire,(3,3),padding='same',strides=(1,1),trainable=True)(x)
    right=ReLU()(right)
    x=concatenate([left,right],axis=3)
    return x

input_shape=(224,224,3)
image_input=Input(shape=input_shape)
x=Conv2D(32,(3,3),strides=(2,2), padding='same',trainable=True)(image_input)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)

x=DepthwiseConv2D((3,3),strides=(1,1),depth_multiplier=1, padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)

x=Conv2D(32,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x1=fire_hee(x,first_filter=144,output_filter=32)
x=Add()([x,x1])
x=Conv2D(144,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)
x=Conv2D(32,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=Activation(relu6)(x)
x=fire_intercept(x,fire=128)
x=Conv2D(32,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)
x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Conv2D(64,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)

x1=fire_hee(x,first_filter=384,output_filter=64)
x=Add()([x,x1])
x=Conv2D(384,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)
x=Conv2D(64,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=Activation(relu6)(x)
x=fire_intercept(x,fire=256)
x=Conv2D(64,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)
x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Conv2D(160,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)

x1=fire_hee(x,first_filter=960,output_filter=160)
x=Add()([x,x1])
x=Conv2D(960,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)
x=Conv2D(160,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=Activation(relu6)(x)
x=fire_intercept(x,fire=640)
x=Conv2D(160,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Activation(relu6)(x)
x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=Conv2D(320,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)


x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)

x=Conv2D(1280,(1,1),strides=(1,1),padding='same',trainable=True)(x)
x=BatchNormalization(trainable=True)(x)
x=ReLU()(x)

x1=GlobalMaxPooling2D()(x)
x1=Dropout(rate=0.5)(x1)
x1=Flatten()(x1)
x1=Dense(512,activation="relu")(x1)
x1=Dropout(0.5)(x1)
out=Dense(3,activation="softmax")(x1)

model=Model(image_input,out)
model.summary()


# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

use relu layer when loading model from disk and customize it layer

In [ ]:
import tensorflow as tf
from keras import backend as K
def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)
tf.keras.utils.get_custom_objects().update({'relu6': relu6})
load_model=load_model('/content/drive/MyDrive/kaggle/scratch_train1.model')


In [ ]:
load_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
load_model.summary()
#layer1=load_model.layers.pop()
#layer2=load_model.layers.pop()
#layer3=load_model.layers.pop()
#layer4=load_model.layers.pop()
#layer5=load_model.layers.pop()
#layer6load_model.layers.pop()
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
#load_model.compile(loss="binary_crossentropy", optimizer=opt,
#	metrics=["accuracy"])
#load_model.summary()
#print(load_model.get_weights)
hidden_layer = load_model.layers[-7].output
# Connect a new layer on it
new_output = ReLU(name="out_relu") (hidden_layer)
# Build a new model
model2 = Model(load_model.input, new_output)
model2.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

ValueError: ignored

copy model to another model with weight

In [ ]:
def copyModel2Model(model_source,model_target,certain_layer=""):        
    for l_tg,l_sr in zip(model_target.layers,model_source.layers):
        wk0=l_sr.get_weights()
        l_tg.set_weights(wk0)
        if l_tg.name==certain_layer:
            break
    print("model source was copied into model target") 
copyModel2Model(load_model,model2,"global_max_pooling2d (GlobalMaxPolling2D")

model source was copied into model target


In [ ]:
model2.get_weights()

[array([[[[-8.96941200e-02,  1.18302532e-01,  9.97705534e-02,
            3.59577104e-03,  8.85985643e-02, -3.25168967e-02,
            1.19054824e-01,  1.33955210e-01,  5.79661876e-02,
           -3.45150530e-02, -6.80415481e-02, -2.48243157e-02,
           -1.06416851e-01,  2.72810869e-02,  8.49731714e-02,
           -3.11969314e-02, -7.36643653e-03,  3.97356711e-02,
            9.89628658e-02,  1.74833480e-02, -5.95926605e-02,
           -5.05232438e-02, -7.86292553e-02,  9.94749516e-02,
           -8.27010348e-03, -9.78466049e-02, -1.15844645e-01,
           -1.16386101e-01,  7.07234740e-02,  1.00654051e-01,
            1.99505016e-02,  2.78683398e-02],
          [-2.74460223e-02, -1.07797891e-01, -3.34191620e-02,
            9.33706462e-02,  1.22424178e-01, -8.97992849e-02,
            3.93425450e-02,  1.36446133e-01, -9.54191759e-02,
            9.27609019e-03,  1.92116713e-03,  5.12418803e-03,
           -3.59350666e-02,  4.95969094e-02, -3.80975939e-02,
           -1.01960011e-

In [ ]:
load_model.get_weights()

[array([[[[-8.96941200e-02,  1.18302532e-01,  9.97705534e-02,
            3.59577104e-03,  8.85985643e-02, -3.25168967e-02,
            1.19054824e-01,  1.33955210e-01,  5.79661876e-02,
           -3.45150530e-02, -6.80415481e-02, -2.48243157e-02,
           -1.06416851e-01,  2.72810869e-02,  8.49731714e-02,
           -3.11969314e-02, -7.36643653e-03,  3.97356711e-02,
            9.89628658e-02,  1.74833480e-02, -5.95926605e-02,
           -5.05232438e-02, -7.86292553e-02,  9.94749516e-02,
           -8.27010348e-03, -9.78466049e-02, -1.15844645e-01,
           -1.16386101e-01,  7.07234740e-02,  1.00654051e-01,
            1.99505016e-02,  2.78683398e-02],
          [-2.74460223e-02, -1.07797891e-01, -3.34191620e-02,
            9.33706462e-02,  1.22424178e-01, -8.97992849e-02,
            3.93425450e-02,  1.36446133e-01, -9.54191759e-02,
            9.27609019e-03,  1.92116713e-03,  5.12418803e-03,
           -3.59350666e-02,  4.95969094e-02, -3.80975939e-02,
           -1.01960011e-

In [ ]:
model2.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
___________________________________________________________________________________________

model training

In [ ]:
# train the head of the network
EPOCHS=20
print("[INFO] training head...")
import tensorflow as tf
from keras import backend as K
def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)
tf.keras.utils.get_custom_objects().update({'relu6': relu6})

#model=load_model('/content/drive/MyDrive/kaggle/scratch_train1.model').summary()
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save('/content/drive/MyDrive/kaggle/scratch_train1.model', save_format="h5")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('/content/drive/MyDrive/kaggle/plot.png')

[INFO] training head...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 32) 0           batch_normalization[0][0]        
____________________________________________________________________

AttributeError: ignored

In [ ]:
model_feat = Model(inputs=model.input,outputs=model.get_layer('dense_3').output)
print(trainX)
print(trainX.shape)
feat_train = model_feat.predict(trainX,batch_size=BS)
print(feat_train.shape)
print(feat_train)
feat_val = model_feat.predict(valX,batch_size=BS)
print(feat_val.shape)

feat_test = model_feat.predict(testX)
print(feat_test.shape)
from sklearn.svm import SVC

svm = SVC(kernel='rbf')

svm.fit(feat_train,np.argmax(trainY,axis=1))

print('fitting done !!!')
svm.score(feat_train,np.argmax(trainY,axis=1))
svm.score(feat_val,np.argmax(valY,axis=1))

In [ ]:
Pred_labels = svm.predict(feat_test)

In [ ]:
#feat_train=feat_train.reshape((feat_train.shape[0],9*9*64))
#print(feat_train.shape[0])

In [ ]:
print(Pred_labels)

In [ ]:
print(testY)

In [ ]:
pip list

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo() #here camera starts
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import numpy as np
import argparse
import cv2
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe('/content/drive/MyDrive/kaggle/deploy.prototxt.txt','/content/drive/MyDrive/kaggle/res10_300x300_ssd_iter_140000.caffemodel')


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os

In [ ]:
from google.colab.patches import cv2_imshow
image = cv2.imread('/content/drive/MyDrive/kaggle/face_test/face2.jpg')

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
model = load_model('/content/drive/MyDrive/kaggle/scratch_train1.model')

# load the input image from disk, clone it, and grab the image spatial
# dimensions

orig = image.copy()
(h, w) = image.shape[:2]

# construct a blob from the image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                             (104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()

# loop over the detections
for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with
    # the detection
    confidence = detections[0, 0, i, 2]

    # filter out weak detections by ensuring the confidence is
    # greater than the minimum confidence
    if confidence > 0.7:
        # compute the (x, y)-coordinates of the bounding box for
        # the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # ensure the bounding boxes fall within the dimensions of
        # the frame
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

        # extract the face ROI, convert it from BGR to RGB channel
        # ordering, resize it to 224x224, and preprocess it
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # pass the face through the model to determine if the face
        # has a mask or not
        (incorrect_mask,with_mask,without_mask) = model.predict(face)[0]

        # determine the class label and color we'll use to draw
        # the bounding box and text
        #hh=model_feat.predict(face)
        #print(hh)
        #k=svm.predict(hh)
        color = (0, 255, 0) 

        # include the probability in the label
        label = "{:.2f}%".format(max(incorrect_mask, with_mask,without_mask) * 100)
        #label="maoo"
        #print(' %d ' % (k))
        print(incorrect_mask, with_mask,without_mask)
        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(image, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

# show the output image
cv2_imshow(image)
#cv2.waitKey(0)

## **Model Play Ground**

In [ ]:
def fire_incept(x, fire=16, intercept=64):
    x = Conv2D(fire, (5,5), strides=(2,2))(x)
    x = LeakyReLU(alpha=0.15)(x)
    
    left = Conv2D(intercept, (3,3), padding='same')(x)
    left = LeakyReLU(alpha=0.15)(left)
    
    right = Conv2D(intercept, (5,5), padding='same')(x)
    right = LeakyReLU(alpha=0.15)(right)
    
    x = concatenate([left, right], axis=3)
    return x

def fire_squeeze(x, fire=16, intercept=64):
    x = Conv2D(fire, (1,1))(x)
    x = LeakyReLU(alpha=0.15)(x)
    
    left = Conv2D(intercept, (1,1))(x)
    left = LeakyReLU(alpha=0.15)(left)
    
    right = Conv2D(intercept, (3,3), padding='same')(x)
    right = LeakyReLU(alpha=0.15)(right)
    
    x = concatenate([left, right], axis=3)
    return x
input_shape=(224,224,3)
image_input=Input(shape=input_shape)

x = fire_incept((image_input), fire=16, intercept=16)

x = fire_incept(x, fire=32, intercept=32)
x = fire_squeeze(x, fire=32, intercept=32)

x = fire_incept(x, fire=64, intercept=64)
x = fire_squeeze(x, fire=64, intercept=64)

x = fire_incept(x, fire=64, intercept=64)
x = fire_squeeze(x, fire=64, intercept=64)

x = Conv2D(64, (3,3))(x)
x = LeakyReLU(alpha=0.1)(x)

x = Flatten()(x)

x = Dense(512)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(0.1)(x)

out = Dense(3, activation='softmax')(x)

model_new = Model(image_input, out)
model_new.summary()

In [ ]:
from keras import backend as K
def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)

def fire_hee(x,first_filter,output_filter):
    x=Conv2D(first_filter,(1,1),strides=(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation(relu6)(x)
    x =DepthwiseConv2D((3,3),strides=(1,1),depth_multiplier=1, padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation(relu6)(x)
    x=Conv2D(output_filter,(1,1),padding='same',strides=(1,1))(x)
    return x


def fire_intercept(x,fire=64):
    left=Conv2D(fire,(1,1),padding='valid',strides=(1,1))(x)
    left=ReLU()(left)

    right=Conv2D(fire,(3,3),padding='same',strides=(1,1))(x)
    right=ReLU()(right)
    x=concatenate([left,right],axis=3)
    return x

input_shape=(224,224,3)
image_input=Input(shape=input_shape)
x=Conv2D(32,(3,3),strides=(2,2), padding='same')(image_input)
x=BatchNormalization()(x)
x=Activation(relu6)(x)

x=DepthwiseConv2D((3,3),strides=(1,1),depth_multiplier=1, padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)

x=Conv2D(32,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x1=fire_hee(x,first_filter=144,output_filter=32)
x=Add()([x,x1])
x=Conv2D(144,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)
x=Conv2D(32,(1,1),strides=(1,1),padding='same')(x)
x=Activation(relu6)(x)
x=fire_intercept(x,fire=128)
x=Conv2D(32,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)
x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same')(x)
x=BatchNormalization()(x)
x=Conv2D(64,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)

x1=fire_hee(x,first_filter=384,output_filter=64)
x=Add()([x,x1])
x=Conv2D(384,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)
x=Conv2D(64,(1,1),strides=(1,1),padding='same')(x)
x=Activation(relu6)(x)
x=fire_intercept(x,fire=256)
x=Conv2D(64,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)
x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same')(x)
x=BatchNormalization()(x)
x=Conv2D(160,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)

x1=fire_hee(x,first_filter=960,output_filter=160)
x=Add()([x,x1])
x=Conv2D(960,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)
x=Conv2D(160,(1,1),strides=(1,1),padding='same')(x)
x=Activation(relu6)(x)
x=fire_intercept(x,fire=640)
x=Conv2D(160,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=Activation(relu6)(x)
x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same')(x)
x=BatchNormalization()(x)
x=Conv2D(320,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)


x=DepthwiseConv2D((3,3),strides=(2,2),depth_multiplier=1, padding='same')(x)
x=BatchNormalization()(x)

x=Conv2D(1280,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)

x=GlobalMaxPooling2D()(x)
x=Dropout(rate=0.5)(x)
x=Flatten()(x)
x=Dense(512,activation="relu")(x)
x=Dropout(0.5)(x)
out=Dense(3,activation="softmax")(x)

model=Model(image_input,out)
model.summary()

backup model

In [ ]:
def fire_hee(x,first_filter=384,output_filter=64):
    x=Conv2D(first_filter,(1,1),strides=(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    x = Conv2D(first_filter, (1, 1), groups=first_filter, strides=(1, 1), padding='same')(x)
    x=BatchNormalization()(x)
    x=ReLU()(x)
    x=Conv2D(output_filter,(1,1),padding='same',strides=(1,1))(x)
    return x


def fire_intercept(x,fire=64):
    left=Conv2D(64,(1,1),padding='same')(x)
    left=ReLU()(left)

    right=Conv2D(64,(3,3),padding='same')(x)
    right=ReLU()(right)
    x=concatenate([left,right],axis=3)
    return x

input_shape=(224,224,3)
image_input=Input(shape=input_shape)
x=Conv2D(32,(3,3),strides=(2,2), padding='same')(image_input)
x=BatchNormalization()(x)
x=ReLU()(x)
x=Conv2D(32,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
x=Conv2D(144,(1,1),strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
x=Conv2D(144,(1,1),groups=144,strides=(1,1),padding='same')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
x=Conv2D(28,(1,1),strides=(1,1),padding='same')(x)
x=fire_intercept(x,fire=64)
x=Conv2D(64,(3,3),strides=(2,2),padding='same')(x)
x=ReLU()(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(5,5),strides=(1,1),padding='same')(x)
x1=fire_hee(x,first_filter=384,output_filter=64)
x=Add()([x,x1])
x=Conv2D(192,(3,3),strides=(2,2),padding='same')(x)
x=ReLU()(x)
x=BatchNormalization()(x)
x=fire_intercept(x,fire=192)
x=Conv2D(384,(1,1),padding='same',strides=(2,2))(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)
x2=fire_hee(x,first_filter=576,output_filter=384)
x=Add()([x,x2])
x=Conv2D(1280,(1,1),strides=(2,2),padding='same')(x)
x=BatchNormalization()(x)
x=ReLU()(x)
x=AveragePooling2D()(x)
x=Dropout(rate=0.5)(x)
x=Flatten()(x)
x=Dense(512,activation="relu")(x)
x=Dropout(0.5)(x)
out=Dense(3,activation="softmax")(x)

model=Model(image_input,out)
model.summary()

In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 16        
_________________________________________________________________
dense_19 (Dense)             (None, 5)                 25        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.models import Model
from keras.layers import Dense, Input

# Assume we have a pretrained model
net_input = Input(shape=(10))
net = Dense(4) (net_input)
net_output = Dense(1) (net)
model1 = Model(net_input, net_output)
model1.summary()
# Get input
new_input = model1.input
# Find the layer to connect
hidden_layer = model1.layers[-2].output
# Connect a new layer on it
new_output = Dense(2) (hidden_layer)
# Build a new model
model2 = Model(new_input, new_output)
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 44        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 44        
______________________________________________